In [1]:
    """
        BERT-pytorch의 corpus 제작
    """
    import json
    import os
    from pre_train import train_tokenizer
    import utils

    from transformers import BertTokenizer
    from plm_dataset import PLMDataset


    config_path = "/root/data/ojt/config/bert_debug.json"
    output_dir = "/root/data/ojt/reference/BERT-pytorch/dataset"
    train_path = "/root/data/ojt/datasets/book_corpus_debug.txt"

    # config_path = "/root/data/ojt/config/bert_debug.json"
    # output_dir = "/root/data/ojt/reference/BERT-pytorch/dataset/books_large_p1_25"
    # train_path = "/root/data/ojt/datasets/books_large_p1_25.txt"
    

    output_path=os.path.join(output_dir,os.path.basename(train_path))
    vocab_file_path = os.path.join(output_dir,utils.VOCAB_TXT_FILE_NAME)

    with open(config_path,"r") as cfg_json:
        config = json.load(cfg_json)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    train_tokenizer(train_path,output_dir,vocab_size=1000)

    
    tokenizer=BertTokenizer(vocab_file=vocab_file_path,do_lower_case=True)
    id_to_vocab={v:k for k,v in tokenizer.get_vocab().items()}
    # print(id_to_vocab)

    plm_dataset=PLMDataset(train_path,tokenizer,config["max_seq_len"],config["max_mask_tokens"],cached_dir=output_dir,use_cache=False)

    with open(output_path,"w") as f:
        first_data=True
        
        for data in plm_dataset:
            # print(data)
            if first_data:
                first_data=False
            else:
                f.write("\n")
            first_input=True
            mask_index=0
            mask_label_list = data["mlm_labels"].tolist()
            mlm_positions = data["mlm_positions"].tolist()
            print(mlm_positions)
            for index, input_id in enumerate(data["input_ids"].tolist()):
                print("input=",id_to_vocab[input_id])
                if not first_input:
                    f.write(" ")
                else:
                    first_input=False

                if index in mlm_positions and index!=0:
                    print(index)
                    print(id_to_vocab[mask_label_list[mask_index]])
                    mask_index+=1
                vocab=id_to_vocab[input_id]
                if vocab=="[SEP]":
                    f.write("\t")
                elif vocab=="[PAD]":
                    break
                elif vocab=="[MASK]":

                    # print(mask_label_list[mask_index])?

                    f.write(id_to_vocab[mask_label_list[mask_index]])

                else:
                    f.write(vocab)
            


    print("Written to %s"%output_path)




Start Loading training data
loading 0/31
Training data successfully loaded
Writing pkl file = /root/data/ojt/reference/BERT-pytorch/dataset/book_corpus_debug.pkl
Successfully written
doc num = 1, sentence num = 31
[2, 5, 10, 21, 27, 32, 36, 42, 61, 63, 64, 68, 75, 85, 105, 106, 107, 123, 0, 0]
input= [CLS]
input= i
input= wi
2
wi
input= ##s
input= ##h
input= [MASK]
5
i
input= ha
input= ##d
input= a
input= be
input= [MASK]
10
##t
input= ##t
input= ##er
input= an
input= ##s
input= ##w
input= ##er
input= to
input= that
input= q
input= ##u
input= [MASK]
21
##es
input= ##t
input= ##ion
input= .
input= star
input= ##l
input= [MASK]
27
##ings
input= ,
input= ne
input= ##w
input= y
input= [MASK]
32
##or
input= ##k
input= is
input= no
input= [MASK]
36
##t
input= the
input= place
input= you
input= ##d
input= e
input= [MASK]
42
##x
input= ##p
input= ##e
input= ##c
input= ##t
input= m
input= ##u
input= ##ch
input= to
input= ha
input= ##pp
input= ##en
input= .
input= [SEP]
input= its
input= a
in

In [12]:
"""모델 불러오기"""
from plm_trainer import PLMDataset
import torch
import json
from model.bert import Bert
from transformers import BertTokenizer
import sys
import utils
from torch.utils.data import DataLoader
import os 

def predict_mask_token(bert,text,with_cuda=False):
    token_result = bert.tokenizer(text, is_split_into_words=True,max_length=bert.config["max_seq_len"],padding="max_length",return_token_type_ids=True)
    # print(token_result)

    for i in token_result["input_ids"]:
        print(id_to_vocab[i],end=" ")

    data ={}

    data["input_ids"] = torch.tensor([token_result["input_ids"]],dtype=torch.int) 
    data["seg_ids"]=torch.tensor([token_result["token_type_ids"]],dtype=torch.int)
    data["att_masks"]=torch.tensor([token_result["attention_mask"]],dtype=torch.int)

    mlm_positions=[]
    mlm_masks=[]
    for index,id in enumerate(token_result["input_ids"]):
        if id ==bert.vocab[utils.MASK_TOKEN]:
            mlm_positions.append(index)
            mlm_masks.append(1)

    if len(mlm_positions)<bert.config["max_mask_tokens"]:
        pad_num = bert.config["max_mask_tokens"]-len(mlm_positions)
        mlm_positions.extend([0]*pad_num)
        mlm_masks.extend([0]*pad_num)
    
    # [max_token_num]
    data["mlm_positions"]=torch.tensor([mlm_positions],dtype=torch.int)
    data["mlm_masks"]=torch.tensor([mlm_masks],dtype=torch.int)

    print(data["mlm_positions"])
    print(data["mlm_masks"])
    
    result,att_score_list = bert(data)
    # print(result)
    print(bert.convert_mask_pred_to_token(result["mask_pred"],data["mlm_masks"],top_k=1))
    print(att_score_list[0][0][0][0])
    # print(att_score_list[0][0][0][1])
    # print(att_score_list[0][0][0][2])

train_path = "/root/data/ojt/datasets/book_corpus_debug.txt"
model_dir = "/root/data/ojt/output/debug_model"
model_name = "checkpoint_2000.pt"

config_path = os.path.join(model_dir,utils.MODEL_CONFIG_NAME)
vocab_file_path = os.path.join(model_dir,utils.VOCAB_TXT_FILE_NAME)
model_path = os.path.join(model_dir,model_name)

# config_path = "/root/data/ojt/config/bert_debug.json"
# vocab_file_path = "/root/data/ojt/output/deprecated/old_scheduler/debug_model_mlm_without_ft/vocab.txt"
# model_path = "/root/data/ojt/output/deprecated/old_scheduler/debug_model_mlm_without_ft/checkpoint_2000.pt"

with open(config_path,"r") as cfg_json:
    config = json.load(cfg_json)

tokenizer=BertTokenizer(vocab_file=vocab_file_path,do_lower_case=True)

bert=Bert(config=config,tokenizer=tokenizer,with_cuda=False,return_mlm=True,return_sop=False)

device = torch.device('cpu')
print("Loading model %s"%model_path)
bert.load_state_dict(torch.load(model_path,map_location=device))
bert.eval()

id_to_vocab = {v:k for k,v in tokenizer.vocab.items()}
plm_dataset=PLMDataset(train_path,tokenizer,128,config["max_seq_len"],config["max_mask_tokens"],cached_dir=model_dir,use_cache=False,mlm_data=True)
plm_data_loader = DataLoader(plm_dataset, batch_size=1,num_workers=1)
    
# for i,data in enumerate(plm_data_loader):
#     # print(data)
#     input_ids=data["input_ids"].view(-1).tolist()
#     for i in input_ids:
#         print(id_to_vocab[i],end=" ")
#     result,att_score_list = bert(data)
#     print("result")
#     print(bert.convert_mask_pred_to_token(result["mask_pred"],data["mlm_masks"],top_k=1))
#     # print(att_score_list
# # while True:
text = "i wish i had a [MASK] answer to that question . starlings , new york is not the place youd expect much to happen . its a small quiet town , the kind where everyone knows your name . its a place where your parents wouldnt even care if you stayed out late biking with your friends ."
predict_mask_token(bert,text)

Loading model /root/data/ojt/output/debug_model/checkpoint_2000.pt
[CLS] i wi ##s ##h i ha ##d a [MASK] an ##s ##w ##er to that q ##u ##es ##t ##ion . star ##l ##ings , ne ##w y ##or ##k is no ##t the place you ##d e ##x ##p ##e ##c ##t m ##u ##ch to ha ##pp ##en . its a s ##m ##all q ##u ##i ##et to ##w ##n , the k ##ind where every ##one know ##s your n ##am ##e . its a place where your p ##are ##nt ##s w ##ou ##ld ##nt e ##v ##en c ##are i ##f you st ##ay ##ed o ##u ##t l ##at ##e b ##i ##k ##ing wi ##th your fr ##i ##en ##ds . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] tensor([[9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=torch.int32)
tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=torch.int32)
[[['be']]]
tensor([0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083,
        0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.0083,
        0.0083, 0.0083, 0.0083, 0.0083, 0.0083, 0.

In [5]:
""" 데이터 제작 확인 """
from transformers import BertTokenizer
from plm_dataset import PLMDataset
from torch.utils.data import DataLoader

vocab_file_path = "/root/data/ojt/output/debug_model/vocab.txt"
train_path = "/root/data/ojt/datasets/book_corpus_debug.txt"

tokenizer=BertTokenizer(vocab_file=vocab_file_path,do_lower_case=True)
plm_dataset=PLMDataset(train_path,tokenizer,data_max_seq_len=64,model_max_seq_len=128,max_mask_tokens=20)
train_data_loader = DataLoader(plm_dataset, batch_size=4,num_workers=1)

vocab = tokenizer.vocab
id_to_vocab={v:k for k,v in vocab.items()}

print(iter(train_data_loader).next())

for data in train_data_loader:
    input_ids=data["input_ids"].tolist()
    mlm_positions=data["mlm_positions"].tolist()
    mlm_labels=data["mlm_labels"].tolist()

     
    for i,seq in enumerate(input_ids):
        mlm_label_index=0
        # print(mlm_positions[i])
        for j,token_id in enumerate(seq):
            

            if j!=0 and j in mlm_positions[i]:
            
                
                print("[M]"+id_to_vocab[mlm_labels[i][mlm_label_index]],end=" ")
                mlm_label_index+=1
            elif token_id!=0:
                print(id_to_vocab[token_id],end=" ")
      

        print("")
    
    break

Reading pkl file = /root/data/ojt/datasets/book_corpus_debug.pkl
doc num = 1, sentence num = 31
{'input_ids': tensor([[  2,  26, 163,  53,   4,  26, 129,  48,  18, 100,  58,  58,  79,  85,
          53,  52,  79,  86,   4,  34,  60, 124,  58, 133,   7,   3, 127,  44,
         174,   5, 158,  52,  42, 140,  64, 118,   4,  58,  78, 188,  99,  48,
          22,  65,   4,  47,   4,  58,  30,   4, 143,  86, 129, 122, 111,   4,
           3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  2, 106,   4, 100, 171, 169,  47, 116, 115,  23,   4,  44,  58, 131,
          36,  49, 174,  47,   5, 131,  20, 141,  72,  55,   7,  78,   4, 153,
 